In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('oecd_data.csv')

In [6]:
df.head()

,country,region,health_exp_pct_gdp,continent,iso3,year,education_exp,public_health_exp,wages_exp,pensions_exp,...,obese_pop_measured,overweight_pop_measured,sugar_supply,tobacco_consumption,total_calories_supply,total_fat_supply,total_protein_supply,vape_pop_15_over,veggie_supply,life_expectancy
0,Australia,Oceania,NaN,Oceania,AUS,1960.0,NaN,NaN,NaN,NaN,...,8.4,28.1,56.2,3576.0,3091.0,111.3,104.9,0.9,61.5,71.00
1,Australia,Oceania,NaN,Oceania,AUS,1961.0,NaN,NaN,NaN,NaN,...,8.4,28.1,56.2,3504.0,3091.0,111.3,104.9,0.9,61.5,71.00
2,Australia,Oceania,NaN,Oceania,AUS,1962.0,NaN,NaN,NaN,NaN,...,8.4,28.1,54.4,3476.0,3113.0,111.8,107.9,0.9,61.4,71.00
3,Australia,Oceania,NaN,Oceania,AUS,1963.0,NaN,NaN,NaN,NaN,...,8.4,28.1,54.8,3456.0,3169.0,112.1,109.1,0.9,60.1,70.96
4,Australia,Oceania,NaN,Oceania,AUS,1964.0,NaN,NaN,NaN,NaN,...,8.4,28.1,56.2,3398.0,3146.0,110.6,110.3,0.9,61.4,70.92
